### 基本概念

- 图（Graph）：图描述了计算的过程，TensorFlow使用图来表示计算任务.    
- 张量（Tensor）：TensorFlow使用tensor表示数据。每个Tensor是一个类型化的多维数组。    
- 操作（op）：图中的节点被称为op（operation的缩写），一个op获得0个或多个Tensor，执行计算，产生0个或多个Tensor。    
- 会话（Session）：图必须在称之为“会话”的上下文中执行。会话将图的op分发到诸如CPU或GPU之类的设备上执行。    
- 变量（Variable）：运行过程中可以被改变，用于维护状态。

### 计算图

#### 构造图
tensorflow程序被组织成为了一个构建阶段和一个执行阶段。其中，构建阶段，op的执行步骤被描述成一个图，所有首先要构建图。    
1. 创建源op（source op）,源op不需要任何输入，例如常量（constant），源op的输出被传递给其他做运算
2. TensorFlow的python库中，op构造器的返回值代表这个op的输出。这些返回值可以作为输入传递给其他op构造器
3. TensorFlow的python库中，包含了默认的graph，可以再上面添加节点

In [6]:
import tensorflow as tf
matrix1 = tf.constant([[3.,3.]])#创建一个常量op，生产一个1x2的矩阵，这个op作为一个节点
matrix2 = tf.constant([[2.],[2.]])#创建一个常量op。产生一个2x1的矩阵
product = tf.matmul(matrix1,matrix2)

#### 启动图
构造阶段完成后，才能在会话中启动图。    
启动图的第一步是创建一个Session对象，如果没有任何参数，会话构造器将启动默认图

In [20]:
with tf.Session() as sess:
    result = sess.run(product)
print(result)

[[12.]]


### Tensor
tensorflow使用tensor数据结构来代表所有的数据。计算图的操作间仅能传递tensor。tensor为多维数组或列表，每一个tensor包含有一个静态类型，一个rank和一个shape。

#### 变量
通常可以将一个统计模型中的参数表示为一组变量。例如，可以讲一个神经网络的权重当做一个tensor储存在变量中，在训练图的重复运行过程中去更细这个tensor

In [23]:
state = tf.Variable(0, name = 'counter')#创建一个变量，并初始化成0
one= tf.constant(1)
new_value = tf.add(state, one)#创建一个op并添加到state
update= tf.assign(state, new_value)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)#在运行了图以后，必须初始化variables
    print(sess.run(state))
    for i in range(3):
        sess.run(update)
        print(sess.run(state))

Instructions for updating:
Colocations handled automatically by placer.
0
1
2
3


#### Fetch
为了取回操作的输出内容，在使用Session对象的run()方法执行图时，传入一些tensor，这些tensor会帮你取回结果。    
之前的例子中，我们之取回了state节点，但是你可以取回多个tensor

In [30]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print(result)

[21.0, 7.0]


#### Feed
上面的例子中展示了在计算图中引入tensor，以常量和变量的形式存储。TensorFlow还提供了feed机制，该机制可以临时替换图中的tensor。

feed使用一个tensor值临时替换一个操作的输出。可以把feed数据作为参数提供给run()方法。标记的方法是使用tf.placeholder()为这些操作创建占位符

In [33]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))


[array([14.], dtype=float32)]
